In [2]:
import pandas as pd
import numpy as np
import random 
random.seed(3)

### GIVEN (1,2) , PFO & OD

In [71]:
seed_number = 4

num_FR = 3
num_option = 6
num_DP = 10

random.seed(seed_number)
def PFO_make(num_FR, num_option):
    PFO = np.zeros([num_FR, num_option])
    for i in range(num_FR):
        num_effected_OP = random.sample([k+1 for k in range(num_option)], k = 1)[0]

        possible_effected_OP = [k for k in range(num_option)]

        effected_OP = random.sample(possible_effected_OP, k=num_effected_OP)
        signed_OP = random.choices([1,-1], weights = (60,40), k=num_effected_OP)

        for j in range(num_effected_OP):
            PFO[i][effected_OP[j]] = signed_OP[j]
    return PFO


def OD_make(num_option, num_DP):
    OD = np.zeros([num_option, num_DP])
    for i in range(num_option):
        num_effected_DP = random.choices([1,2,3], weights=(60, 30, 10), k=1)[0]

        possible_effected_DP = [k for k in range(num_DP)]

        effected_DP = random.sample(possible_effected_DP, k=num_effected_DP)
        signed_DP = random.choices([1,-1], weights = (60,40), k=num_effected_DP)

        for j in range(num_effected_DP):
            OD[i][effected_DP[j]] = signed_DP[j]
    return OD

random.seed(seed_number)

PFO = PFO_make(num_FR, num_option)
OD = OD_make(num_option, num_DP)

FR_index = ["FR_{}".format(i+1) for i in range(num_FR)]
Opt_index = ["Opt_{}".format(i+1) for i in range(num_option)]
DP_index = ["DP_{}".format(i+1) for i in range(num_DP)]


In [72]:
pd.DataFrame(PFO, index = FR_index, columns = Opt_index).astype(int)

,Opt_1,Opt_2,Opt_3,Opt_4,Opt_5,Opt_6
FR_1,1,0,-1,0,0,0
FR_2,1,0,0,0,0,0
FR_3,1,1,1,-1,0,1


In [73]:
pd.DataFrame(OD, index = Opt_index, columns = DP_index).astype(int)

,DP_1,DP_2,DP_3,DP_4,DP_5,DP_6,DP_7,DP_8,DP_9,DP_10
Opt_1,0,0,0,0,1,0,0,0,0,1
Opt_2,0,1,0,0,0,-1,0,0,0,0
Opt_3,0,0,0,1,0,0,0,0,0,0
Opt_4,0,-1,0,0,0,0,0,0,0,0
Opt_5,0,0,0,0,1,0,0,0,1,0
Opt_6,0,0,0,0,1,0,0,0,1,0


### GIVEN 3 , PM

In [163]:
random.seed(seed_number)

# 각 DP 마다 path 만들기
def path_make(DP_index):
    num_path = random.randint(1,3)
    path_DP = np.zeros((num_path, 1, num_DP))
    
    for i in range(len(path_DP)):
        possible_effected_DP = [k for k in range(num_DP) if k != DP_index]
        num_effected_DP = random.choices([0,1,2,3], weights=(5, 35, 50, 10), k=1)[0]
        effected_DP = random.sample(possible_effected_DP, k=num_effected_DP)
        signed_DP = random.choices([1,-1], weights = (60,40), k=num_effected_DP)
        
        
        for j in range(num_effected_DP):
            path_DP[i][0][effected_DP[j]] = signed_DP[j]
        
    return path_DP

# 각 path중 하나를 골라서 path_matrix 만들기
def make_PM(combination_index):
    empty_list = []
    for i in range(len(combination_index)):
        empty_list.append(combination_index[i][0])
    
    return np.matrix(empty_list)    

random.seed(seed_number)
path_DP_list = []
for i in range(num_DP):
    num_path = random.randint(1,3)
    path_DP_list.append(path_make(i))
    
import itertools

# 가능한 모든 조합 생성
combinations = list(itertools.product(*path_DP_list))

# PM matrix 만들기
PM_list = []
for i in combinations:
    PM_list.append(make_PM(i))
    

In [164]:
#ex) first PM_list 
pd.DataFrame(PM_list[0], index = DP_index, columns = DP_index).astype(int)

,DP_1,DP_2,DP_3,DP_4,DP_5,DP_6,DP_7,DP_8,DP_9,DP_10
DP_1,0,0,0,0,0,0,0,1,0,0
DP_2,0,0,0,0,0,0,0,0,0,1
DP_3,-1,0,0,0,0,0,0,0,0,0
DP_4,0,0,0,0,0,1,0,0,0,0
DP_5,0,0,0,1,0,0,0,0,0,0
DP_6,0,0,0,1,1,0,0,0,0,0
DP_7,0,0,0,0,-1,0,0,0,1,0
DP_8,0,0,0,0,0,0,-1,0,0,0
DP_9,0,0,0,-1,-1,0,0,0,0,0
DP_10,0,0,1,0,0,0,0,0,0,0


### phase 1

In [165]:
#phase 1-1 FR - Option
Z = []
for i in range(len(PFO)):
    Z.append([int((k+1)*PFO[i][k]) for k in range(len(PFO[i])) if PFO[i][k] != 0])
combinations = list(itertools.product(*Z))

Possible_Option_Candidates = []
for k in combinations:
    kk = list(k)
    kkk = [abs(j) for j in kk ]
    if len(kkk) == len(set(kkk)):
        Possible_Option_Candidates.append(kk)
Possible_Option_Candidates

[[-3, 1, 2], [-3, 1, -4], [-3, 1, 6]]

In [166]:
#phase 1-2 Option - DP 
def checking_DP(Option_candidates):
    total_DP = []
    for i in range(num_DP):
        negative_sign = 0
        positive_sign = 0

        for j in Option_candidates:
            if np.sign(j) * OD[abs(j)-1][i] == -1:
                negative_sign += 1
            elif np.sign(j) * OD[abs(j)-1][i] == 1 :
                positive_sign += 1
            
        if (negative_sign >0 ) and (positive_sign > 0):
            print("conflict exists on DP-level", Option_candidates, ", DP : ", i+1)
            return False, False
        
        elif (negative_sign ==0) and (positive_sign == 0):
            total_DP.append(0)
        elif negative_sign>0:
            total_DP.append(-1)
        else:
            total_DP.append(1)
            
        
    return total_DP, Option_candidates

Initated_DP_Candidates = []
Initated_OP_Candidates = []
for i in Possible_Option_Candidates:
    A,B = checking_DP(i)
    if A != False:
        Initated_DP_Candidates.append(A)
        Initated_OP_Candidates.append(B)
print("__________________________________")
print("DP LIST : ", Initated_DP_Candidates)
print("")
print("OP LIST : ", Initated_OP_Candidates)            

__________________________________
DP LIST :  [[0, 1, 0, -1, 1, -1, 0, 0, 0, 1], [0, 1, 0, -1, 1, 0, 0, 0, 0, 1], [0, 0, 0, -1, 1, 0, 0, 0, 1, 1]]

OP LIST :  [[-3, 1, 2], [-3, 1, -4], [-3, 1, 6]]


### Phase 2 DP - propagation

In [167]:
def one_hot_encoding(DP_list):
    fulllist = []
    for k in range(len(DP_list)):
        if DP_list[k] != 0 :
            imme_list = np.zeros(len(DP_list))
            imme_list[k] = DP_list[k]
            fulllist.append(imme_list)
    return fulllist
            
def conflict_find(Initated_DP_Candidates):
    conflict_list = []
    for i in range(len(Initated_DP_Candidates)):
        DP_list = one_hot_encoding(Initated_DP_Candidates[i])
        path_list = []
        for j in range(len(DP_list)):
            print(DP_list[j])
            first_prop = []
            second_prop = []
            third_porp = []
            imme_list = []

            imme_list.append(DP_list[j])

            for i in PM_list:
                first_prop.append(np.dot(DP_list[j], i))
            first_prop = np.unique(first_prop, axis=0).reshape(-1)
            print(first_prop)
            imme_list.append(first_prop)

            for first in [first_prop]: 
                for i in PM_list:
                    second_prop.append(np.dot(first, i))
                second_prop = np.unique(second_prop, axis=0).reshape(-1)
            print(second_prop)
            imme_list.append(second_prop)

            for second in [second_prop]: 
                for i in PM_list:
                    third_porp.append(np.dot(second, i))
                third_porp = np.unique(third_porp, axis=0).reshape(-1)  
            print(third_porp)
            imme_list.append(third_porp)
            path_list.append(imme_list)
            print("_____________________________________________")

        Full_total = []
        for j in range(len(path_list)):
            Total = []
            for i in range(len(path_list[0][0])):
                total_index = path_list[j][0][i] + path_list[j][1][i] + path_list[j][2][i] + path_list[j][3][i]

                Total.append(np.sign(total_index))
            Full_total.append(Total)
        conflict_degree = 0
        for i in range(len(Full_total[0])):
            negative_sign = 0
            positive_sign = 0

            for j in range(len(Full_total)):
                if np.sign(Full_total[j][i]) == -1:
                    negative_sign += 1
                elif np.sign(Full_total[j][i])  == 1 :
                    positive_sign += 1
            print("positive_sign : " , positive_sign)
            print("negative_sign : ", negative_sign)

            print("_________________")
            if (negative_sign >0 ) and (positive_sign > 0):
                conflict_degree +=1
        print("conflcit_degree :", conflict_degree)
        conflict_list.append(conflict_degree)
    
    return conflict_list
     

In [168]:
conflict_find(Initated_DP_Candidates)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
_____________________________________________
[ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
[ 0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.]


ValueError: shapes (20,) and (10,10) not aligned: 20 (dim 0) != 10 (dim 0)

In [33]:
len(Initated_DP_Candidates)

12

In [17]:
DP_list = one_hot_encoding(Initated_DP_Candidates[1])

In [169]:
DP_list = one_hot_encoding(Initated_DP_Candidates[0])

path_list = []
for j in range(len(DP_list)):
    print(DP_list[j])
    first_prop = []
    second_prop = []
    Second_prop = []
    
    third_prop = []
    Third_prop = []
    imme_list = []
    
    imme_list.append(DP_list[j])

    for i in PM_list:
        first_prop.append(np.dot(DP_list[j], i))
    first_prop = np.unique(first_prop, axis=0).reshape(-1)
    print(first_prop)
    imme_list.append(first_prop)

    for first in [first_prop]: 
        for i in PM_list:
            second_prop.append(np.dot(first, i))
        second_prop = np.unique(second_prop, axis=0).reshape(-1)
        Second_prop.append(second_prop)
    print(Second_prop)
    imme_list.append(second_prop)

    for second in [second_prop]: 
        for i in PM_list:
            third_prop.append(np.dot(second, i))
        third_prop = np.unique(third_prop, axis=0).reshape(-1)  
        Third_prop.append(third_prop)
        
    print(Third_prop)
    imme_list.append(third_porp)
    path_list.append(imme_list)
    print("_____________________________________________")

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])]
[array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
_____________________________________________
[ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
[ 0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.]


ValueError: shapes (20,) and (10,10) not aligned: 20 (dim 0) != 10 (dim 0)

In [107]:
Full_total = []
for j in range(len(path_list)):
    Total = []
    for i in range(len(path_list[0][0])):
        total_index = path_list[j][0][i] + path_list[j][1][i] + path_list[j][2][i] + path_list[j][3][i]
        
        Total.append(np.sign(total_index))
    Full_total.append(Total)
conflict_degree = 0
for i in range(len(Full_total[0])):
    negative_sign = 0
    positive_sign = 0

    for j in range(len(Full_total)):
        if np.sign(Full_total[j][i]) == -1:
            negative_sign += 1
        elif np.sign(Full_total[j][i])  == 1 :
            positive_sign += 1
    print("positive_sign : " , positive_sign)
    print("negative_sign : ", negative_sign)
    
    print("_________________")
    if (negative_sign >0 ) and (positive_sign > 0):
        conflict_degree +=1
print("conflcit_degree :", conflict_degree)

positive_sign :  0
negative_sign :  1
_________________
positive_sign :  1
negative_sign :  0
_________________
positive_sign :  2
negative_sign :  0
_________________
positive_sign :  0
negative_sign :  4
_________________
positive_sign :  1
negative_sign :  2
_________________
positive_sign :  0
negative_sign :  4
_________________
positive_sign :  0
negative_sign :  0
_________________
positive_sign :  0
negative_sign :  0
_________________
positive_sign :  0
negative_sign :  0
_________________
positive_sign :  2
negative_sign :  0
_________________
conflcit_degree : 1


In [30]:
Full_total

[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, -1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, -1.0, 1.0, 1.0]]

In [138]:
path_DP_list

[array([[[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]],
 
        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]]),
 array([[[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]]),
 array([[[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.]]]),
 array([[[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]]),
 array([[[ 0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]]),
 array([[[ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.]],
 
        [[ 0., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.]]]),
 array([[[ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]]]),
 array

In [384]:
for j in [0]:
    print(DP_list[j])
    first_prop = []
    Second_prop = []
    Third_prop = []
    
    imme_list = []
    imme_list.append(DP_list[j])

    for i in PM_list:
        first_prop.append(np.dot(DP_list[j], i))
    
    first_prop = np.unique(first_prop, axis=0)
    first_prop = first_prop.reshape(len(first_prop),-1)
    
    for first in first_prop: 
        second_prop = []

        for i in PM_list:
            second_prop.append(np.dot(first, i))

        second_prop = np.unique(second_prop, axis=0)

        second_prop = second_prop.reshape(len(second_prop),-1)

        Second_prop.append(second_prop)
            
            
    Third_prop = []

    for Second in Second_prop:
        for second in Second:
            third_prop = []

            for i in PM_list:
                third_prop.append(np.dot(second, i))

            third_prop = np.unique(third_prop, axis=0)

            third_prop = third_prop.reshape(len(third_prop),-1)

            Third_prop.append(third_prop)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [300]:

Second_prop = []
for first in first_prop: 
    second_prop = []
    
    for i in PM_list:
        second_prop.append(np.dot(first, i))
    
    second_prop = np.unique(second_prop, axis=0)
   
    second_prop = second_prop.reshape(len(second_prop),-1)
    
    if Second_prop == []:
        Second_prop.append(second_prop)
    else:
        Second_prop = np.concatenate((Second_prop[0], second_prop), axis = 0)
    
    
    print("second : ", second_prop)

second :  [[ 0. -1.  0.  0.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  1. -1.]]
second :  [[ 0.  0. -1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [170]:

DP_list = one_hot_encoding(Initated_DP_Candidates[0])
path_list = []
for j in [1]:
    print(DP_list[j])
    first_prop = []
    second_prop = []
    third_porp = []
    imme_list = []
    imme_list.append(DP_list[j])

    for i in PM_list:
        first_prop.append(np.dot(DP_list[j], i))
    
    first_prop = np.unique(first_prop, axis=0).reshape(-1)
    print(first_prop)
    imme_list.append(first_prop)

    for first in [first_prop]: 
        for i in PM_list:
            second_prop.append(np.dot(first, i))
        
        second_prop = np.unique(second_prop, axis=0).reshape(-1)

    imme_list.append(second_prop)

    for second in [second_prop]: 
        for i in PM_list:
            third_porp.append(np.dot(second, i))
        #print(third_porp)
        third_porp = np.unique(third_porp, axis=0).reshape(-1)  
            

[ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
[ 0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.]


ValueError: shapes (20,) and (10,10) not aligned: 20 (dim 0) != 10 (dim 0)